In [25]:
import time
from datetime import datetime, timedelta
import vk_api
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json


def get_group_ids(file_path):
    with open(file_path, 'r') as file:
        group_ids = [line.strip() for line in file]
    return group_ids


def get_or_create_worksheet(spreadsheet, worksheet_name):
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=100, cols=30)
    return worksheet


def collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, max_post_count, token):
    vk_session = vk_api.VkApi(token=token)
    api = vk_session.get_api()

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../access/credentials.json', scope)
    client = gspread.authorize(credentials)
    target_spreadsheet = client.open_by_key(target_spreadsheet_key)

    worksheet_name = group_id
    worksheet = get_or_create_worksheet(target_spreadsheet, worksheet_name)

    start_date = datetime.now().date() - timedelta(days=num_days)
    rows_to_append = []

    offset = 0
    post_count = 0

    while True:
        posts = api.wall.get(owner_id=group_id, count=100, offset=offset, extended=1)['items']
        if not posts:
            break

        sorted_posts = sorted(posts, key=lambda x: x['date'], reverse=True)

        for post in sorted_posts:
            post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')

            if datetime.strptime(post_date, '%Y-%m-%d %H:%M:%S').date() >= start_date:
                post_link = f"https://vk.com/wall-{group_id[1:]}_{post['id']}"
                text = post['text']
                comments = post.get('comments', {}).get('count', 0)
                likes = post.get('likes', {}).get('count', 0)
                reposts = post.get('reposts', {}).get('count', 0)
                views = post.get('views', {}).get('count', 0)

                data = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments,
                        likes, reposts, views]

                rows_to_append.append(data)

                post_count += 1

                if post_count >= max_post_count:
                    break

        if post_count >= max_post_count:
            break

        offset += 100

    if rows_to_append:
        worksheet.clear()
        worksheet.update('A1:H1', [['Дата и время проверки', 'Дата публикации', 'Ссылка на пост', 'Текст', 'Комментарии',
                                    'Лайки', 'Репосты', 'Весь охват']])
        worksheet.append_rows(rows_to_append[::-1], value_input_option='USER_ENTERED')
        print(f'[{datetime.now()}] Записано {len(rows_to_append)} строк для {group_id}.')
    else:
        print(f'[{datetime.now()}] Нет данных для записи в таблицу.')


if __name__ == "__main__":
    with open('../../../access/config.json') as config_file:
        config = json.load(config_file)

    target_spreadsheet_key = config['target_spreadsheet_key']
    num_days = config['num_days']
    min_post_count = config['min_post_count']
    max_post_count = config['max_post_count']
    token = config['vk_token_My']

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../access/credentials.json', scope)
    client = gspread.authorize(credentials)

    group_ids = get_group_ids('../../../access/groups.txt')

    for group_id in group_ids:
        collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, max_post_count, token)


[2023-07-02 18:19:49.665854] Записано 3 строк для -193643180.
[2023-07-02 18:19:53.207172] Записано 5 строк для -221114738.
[2023-07-02 18:19:58.469813] Записано 65 строк для -28473421.
[2023-07-02 18:20:17.123848] Записано 522 строк для -35217360.
[2023-07-02 18:20:42.862219] Записано 1060 строк для -53902978.
